<a href="https://colab.research.google.com/github/annisatwain/TugasAPI/blob/main/Question_answer_rag_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch torchvision torchaudio scikit-learn -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
from transformers import __version__ as transformers_version
from datasets import __version__ as datasets_version
print(torch.__version__)
print(torch.cuda.is_available())  # True jika GPU tersedia
print(transformers_version)
print(datasets_version)


2.5.1+cu121
True
4.47.1
3.2.0


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from transformers import pipeline
import torch

# 1. Load Dataset
dataset = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")

# 2. Split dataset into train and test (use `train_test_split` from `datasets`)
split_dataset = dataset["test"].train_test_split(test_size=0.2, seed=42)
data_train = split_dataset["train"]
data_test = split_dataset["test"]

# 3. Convert dataset into appropriate format for QA
train_contexts, train_questions, train_answers = [], [], []
for row in data_train:
    train_questions.append(row["question"])
    train_answers.append(row["answer"])
    train_contexts.append(row["question"] + " " + row["answer"])

test_contexts, test_questions, test_answers = [], [], []
for row in data_test:
    test_questions.append(row["question"])
    test_answers.append(row["answer"])
    test_contexts.append(row["question"] + " " + row["answer"])

# 4. Initialize Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Function to add token positions
def add_token_positions(encodings, answers, tokenizer):
    start_positions = []
    end_positions = []
    for i, answer in enumerate(answers):
        # Find start and end positions
        start_idx = encodings.char_to_token(i, 0)
        end_idx = encodings.char_to_token(i, len(answer) - 1)

        # If answer is not found, set to max length
        if start_idx is None:
            start_idx = tokenizer.model_max_length
        if end_idx is None:
            end_idx = tokenizer.model_max_length

        start_positions.append(start_idx)
        end_positions.append(end_idx)

    encodings.update({
        'start_positions': start_positions,
        'end_positions': end_positions
    })
    return encodings

# 5. Tokenize and preprocess the data
def preprocess(data_contexts, data_questions, data_answers):
    tokenized = tokenizer(
        data_contexts,
        data_questions,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"
    )
    # Add start and end positions
    add_token_positions(tokenized, data_answers, tokenizer)
    return tokenized

train_data = preprocess(train_contexts, train_questions, train_answers)
test_data = preprocess(test_contexts, test_questions, test_answers)

# Dataset class
def convert_to_tensor(data, idx):
    return torch.tensor(data[idx])

class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": convert_to_tensor(self.encodings["input_ids"], idx),
            "attention_mask": convert_to_tensor(self.encodings["attention_mask"], idx),
            "start_positions": convert_to_tensor(self.encodings["start_positions"], idx),
            "end_positions": convert_to_tensor(self.encodings["end_positions"], idx),
        }

train_dataset = QADataset(train_data)
test_dataset = QADataset(test_data)

# 6. Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 7. Train the Model
trainer.train()

# 8. Evaluate the Model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

# 9. Test Model with Pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)
example = {
    "question": "Was Abraham Lincoln the sixteenth President of the United States?",
    "context": "Abraham Lincoln was the 16th president of the United States. He led the country during the Civil War."
}
result = qa_pipeline(example)
print("Prediction:", result)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/54.4k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/918 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss
1,2.124400,1.694163
2,0.815600,0.813125
3,0.517300,0.777049
4,0.403000,0.911768
5,0.333100,1.146687
6,0.042700,1.705627
7,0.097000,1.682090
8,0.143300,1.545079
9,0.128800,1.542085
10,0.048900,1.667253


<ipython-input-3-a07ff464ea92>:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(data[idx])
<ipython-input-3-a07ff464ea92>:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(data[idx])
<ipython-input-3-a07ff464ea92>:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(data[idx])
<ipython-input-3-a07ff464ea92>:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch

Device set to use cuda:0


Evaluation Results: {'eval_loss': 0.7770485281944275, 'eval_runtime': 4.816, 'eval_samples_per_second': 38.206, 'eval_steps_per_second': 4.776, 'epoch': 50.0}
Prediction: {'score': 0.02016940712928772, 'start': 61, 'end': 90, 'answer': 'He led the country during the'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [ ]:
# 10. Experiment with Custom Questions
custom_example_1 = {
    "question": "Who was Abraham Lincoln?",
    "context": "Abraham Lincoln was the 16th president of the United States. He led the country during the Civil War. He is known for abolishing slavery and promoting unity."
}
custom_result_1 = qa_pipeline(custom_example_1)
print("Custom Prediction 1:", custom_result_1)

custom_example_2 = {
    "question": "What is Abraham Lincoln known for?",
    "context": "Abraham Lincoln was the 16th president of the United States. He led the country during the Civil War. He is known for abolishing slavery and promoting unity."
}
custom_result_2 = qa_pipeline(custom_example_2)
print("Custom Prediction 2:", custom_result_2)


Custom Prediction 1: {'score': 0.004176327493041754, 'start': 105, 'end': 113, 'answer': 'is known'}
Custom Prediction 2: {'score': 0.0032127031590789557, 'start': 102, 'end': 150, 'answer': 'He is known for abolishing slavery and promoting'}


In [ ]:
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.txt',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')

In [ ]:
import torch

# Simpan model dalam format .pth
torch.save(model.state_dict(), "./final_model/model.pth")

print("Model has been saved as model.pth")


Model has been saved as model.pth


In [ ]:
from transformers import AutoModelForQuestionAnswering

# Load model architecture
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Load saved weights
model.load_state_dict(torch.load("./final_model/model.pth"))

print("Model has been loaded from model.pth")


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-13-4cc6fd596f4e>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend

Model has been loaded from model.pth
